# Le modèle de données


## Introduction

Laravel inclut Eloquent, un **ORM** qui rend agréable l'interaction avec votre base de données. Lorsque vous utilisez Eloquent, chaque table de base de données a un "modèle" correspondant qui est utilisé pour interagir avec cette table. En plus de récupérer des enregistrements de la table de la base de données, les modèles Eloquent vous permettent d'insérer, de mettre à jour et de supprimer des enregistrements de la table.

### Configuration

La configuration des services de base de données de Laravel se trouve dans le fichier de configuration config/database.php de votre application. Dans ce fichier, vous pouvez définir toutes vos connexions à la base de données, ainsi que spécifier quelle connexion doit être utilisée par défaut. La plupart des options de configuration de ce fichier sont déterminées par les valeurs des variables d'environnement de votre application. Des exemples pour la plupart des systèmes de bases de données supportés par Laravel sont fournis dans ce fichier.

Par défaut, l'exemple de configuration d'environnement de Laravel est prêt à être utilisé avec Laravel Sail, qui est une configuration Docker pour développer des applications Laravel sur votre machine locale. Cependant, vous êtes libre de modifier votre configuration de base de données en fonction des besoins de votre base de données locale

## Création du modèle

Pour commencer, nous allons créer un modèle Eloquent. Les modèles se trouvent généralement dans le répertoire `app\Models` et héritent de la classe `Illuminate\Database\Eloquent\Model`. Vous pouvez utiliser la commande  `make:model` pour générer un nouveau modèle :

In [ ]:
%%bash

php artisan make:model Product

Si vous souhaitez générer une migration de la base de données lorsque vous générez le modèle, vous pouvez utiliser l'option --migration ou -m :

In [ ]:
%%bash

php artisan make:model Product --migration

Vous pouvez générer divers autres types de classes lors de la génération d'un modèle, comme les usines, les ensemenceurs, les politiques, les contrôleurs et les requêtes de formulaire. En outre, ces options peuvent être combinées pour créer plusieurs classes à la fois :

In [ ]:
%%bash

# Générer un modèle et une classe FlightFactory...
php artisan make:model Product --factory

# Générer un modèle et une classe FlightSeeder...
php artisan make:model Product --seed
 
# Génère un modèle et une classe FlightController...
php artisan make:model Product --controller
 
# Génère un modèle, une classe ressource FlightController, et des classes de requête de formulaire...
php artisan make:model Product --controller --resource --requests
 
# Générer un modèle et une classe FlightPolicy...
php artisan make:model Product --policy
 
# Génère un modèle et une classe de migration, factory, seeder, et controller...
php artisan make:model Product -mfsc
 
# Raccourci pour générer un modèle, une migration, une usine, un seeder, une politique, un contrôleur et des demandes de formulaire...
php artisan make:model Product --all
 
# Générer un modèle de pivot...
php artisan make:model Member --pivot

### Inspection des modèles

Parfois, il peut être difficile de déterminer tous les attributs et relations disponibles d'un modèle juste en parcourant son code. Essayez plutôt la commande artisan model:show, qui fournit un aperçu pratique de tous les attributs et relations du modèle :

In [ ]:
%%bash

php artisan model:show Product

### Configuration du modèle

Les modèles générés par la commande `make:model` seront placés dans le répertoire `app/Models`. Examinons une classe de modèle de base et discutons de certaines des conventions clés d'Eloquent :

In [ ]:
%%php
<?php
 
espace de noms App\Models ;
 
utilisez Illuminate\Database\Eloquent\Model ;
 
class Product extends Model
{
    //
}

#### Noms de table

Après avoir jeté un coup d'œil à l'exemple ci-dessus, vous avez peut-être remarqué que nous n'avons pas indiqué à Eloquent quelle table de base de données correspond à notre modèle `Product`. Par convention, le nom en "snake case", de la classe au pluriel sera utilisé comme nom de table à moins qu'un autre nom ne soit explicitement spécifié. Ainsi, dans ce cas, Eloquent supposera que le modèle `Product` stocke les enregistrements dans la table `products`, tandis qu'un modèle `ProductLine` stockerait les enregistrements dans la table `product_lines`.

Si la table de base de données correspondante de votre modèle ne correspond pas à cette convention, vous pouvez spécifier manuellement le nom de la table du modèle en définissant une propriété de table sur le modèle :

In [ ]:
%%php

/**
 * @var string
 */
protected $table = 'my_products' ;

#### Clés primaires

Eloquent suppose également que la table de base de données correspondante de chaque modèle possède une colonne de clé primaire nommée `id`. Si nécessaire, vous pouvez définir une propriété protégée `primaryKey` sur votre modèle pour spécifier une colonne différente qui sert de clé primaire à votre modèle :

In [ ]:
%%php

/**
 * @var string
 */
protected $primaryKey = 'productCode' ;

De plus, Eloquent suppose que la clé primaire est une valeur entière incrémentale, ce qui signifie qu'Eloquent va automatiquement convertir la clé primaire en un entier. Si vous souhaitez utiliser une clé primaire non incrémentale ou non numérique, vous devez définir une propriété publique `incrementing` sur votre modèle qui est définie à `false`. i la clé primaire de votre modèle n'est pas un nombre entier, vous devez définir une propriété protégée $keyType sur votre modèle. Cette propriété doit avoir une valeur de chaîne de caractères.

In [ ]:
%%php

/**
 * @var bool
 */
public $incrementing = false ;

/**
 * @var string
 */
protected $keyType = 'string' ;


##### Clés UUID et ULID

Au lieu d'utiliser des entiers auto-incrémentés comme clés primaires de votre modèle Eloquent, vous pouvez choisir d'utiliser des UUIDs. Les UUID sont des identifiants alphanumériques universellement uniques de 36 caractères.

Si vous souhaitez qu'un modèle utilise une clé UUID au lieu d'une clé entière à incrémentation automatique, vous pouvez utiliser le trait `Illuminate\Database\Eloquent\Concerns\HasUuids` sur le modèle. Bien entendu, vous devez vous assurer que le modèle possède une colonne de clé primaire équivalente à UUID :

In [ ]:
%%php

use Illuminate\Database\Eloquent\Concerns\HasUuids ;
use IlluminateDatabase\Eloquent\Model ;
 
class Article extends Model
{
    use HasUuids ;
 
    // ...
}
 
$article = Article::create(['title' => 'Traveling to Europe']) ;
 
$article->id ; // " 8f8e8478-9035-4d23-b9a7-62f4d2612ce5 "

Par défaut, le trait HasUuids génère des UUID "ordonnés" pour vos modèles. Ces UUID sont plus efficaces pour le stockage dans des bases de données indexées car ils peuvent être triés lexicographiquement.

Vous pouvez remplacer le processus de génération des UUID pour un modèle donné en définissant une méthode `newUniqueId` sur le modèle. En outre, vous pouvez préciser quelles colonnes doivent recevoir des UUID en définissant une méthode `uniqueIds` sur le modèle :

In [ ]:
%%php

use Ramsey\Uuid\Uuid ;
 
/**
 * Génère un nouvel UUID pour le modèle.
 *
 * @return string
 */
public function newUniqueId()
{
    return (string) Uuid::uuid4() ;
}
 
/**
 * Obtenez les colonnes qui doivent recevoir un identifiant unique.
 *
 * @return array
 */
public function uniqueIds()
{
    return ['id', 'discount_code'] ;
}

Si vous le souhaitez, vous pouvez choisir d'utiliser des "ULID" au lieu des UUID. Les ULID sont similaires aux UUID, mais ils ne comportent que 26 caractères. Comme les UUID ordonnés, les ULID peuvent être triés lexicographiquement pour une indexation efficace de la base de données. Pour utiliser les ULID, vous devez utiliser le trait Illuminate\Database\Eloquent\Concerns\HasUlids sur votre modèle. Vous devez également vous assurer que le modèle possède une colonne de clé primaire équivalente aux ULID :

In [ ]:
%%php

use IlluminateDatabase\Eloquent\Concerns\HasUlids ;
use Illuminate\Database\Eloquent\Model ;
 
class Article extends Model
{
    use HasUlids ;
 
    // ...
}
 
$article = Article::create(['title' => 'Traveling to Asia']) ;
 
$article->id ; // "01gd4d3tgrrfqeda94gdbtdk5c"

#### Horodatage

Par défaut, Eloquent s'attend à ce que les colonnes created_at et updated_at existent sur la table de base de données correspondante de votre modèle. Eloquent définit automatiquement les valeurs de ces colonnes lorsque les modèles sont créés ou mis à jour. Si vous ne voulez pas que ces colonnes soient automatiquement gérées par Eloquent, vous devez définir une propriété $timestamps sur votre modèle avec une valeur de false :

In [ ]:
%%php

/**
 * @var bool
 */
public $timestamps = false ;

Si vous devez personnaliser le format des horodatages de votre modèle, définissez la propriété `dateFormat` sur votre modèle. Cette propriété détermine la façon dont les attributs de date sont stockés dans la base de données, ainsi que leur format lorsque le modèle est sérialisé dans un tableau ou JSON :

In [ ]:
%%php
 
/**
 * @var string
 */
protected $dateFormat = 'U' ;


Si vous avez besoin de personnaliser les noms des colonnes utilisées pour stocker les horodatages, vous pouvez définir les constantes CREATED_AT et UPDATED_AT sur votre modèle :

In [ ]:
%%php
 
class Product extends Model
{
    const CREATED_AT = 'creation_date' ;
    const UPDATED_AT = 'updated_date' ;
}

Si vous souhaitez effectuer des opérations sur le modèle sans que l'horodatage de la date de mise à jour du modèle ne soit modifié, vous pouvez opérer sur le modèle dans une fermeture donnée à la méthode withoutTimestamps :

In [ ]:
%%php

Model::withoutTimestamps(fn () => $post->increment(['reads'])) ;

#### Connexion aux bases de données

Par défaut, tous les modèles Eloquent utiliseront la connexion de base de données par défaut qui est configurée pour votre application. Si vous souhaitez spécifier une connexion différente qui doit être utilisée lors de l'interaction avec un modèle particulier, vous devez définir une propriété $connection sur le modèle :

In [ ]:
%%php
 
namespace App\Models ;
 
use Illuminate\Database\Eloquent\Model ;
 
class Product extends Model
{
    /**
     * La connexion à la base de données qui doit être utilisée par le modèle.
     *
     * @var string
     */
    protected $connection = 'sqlite' ;
}

#### Valeurs d'attribut par défaut

Par défaut, une instance de modèle nouvellement instanciée ne contient aucune valeur d'attribut. Si vous souhaitez définir les valeurs par défaut de certains attributs de votre modèle, vous pouvez définir une propriété $attributes sur votre modèle. Les valeurs d'attributs placées dans le tableau $attributes doivent être dans leur format brut, "stockable", comme si elles venaient d'être lues dans la base de données :

In [ ]:
%%php
 
namespace App\Models ;
 
use Illuminate\Database\Eloquent\Model ;
 
class product extends Model
{
    /**
     * Les valeurs par défaut du modèle pour les attributs.
     *
     * @var array
     */
    protected $attributes = [
        'scale' => '1:10',
        'productVendor' => 'Heller',
    ] ;
}

#### Configuration de la rigueur d'Eloquent

Laravel offre plusieurs méthodes qui vous permettent de configurer le comportement et la "sévérité" d'Eloquent dans une variété de situations.

Premièrement, la méthode `preventLazyLoading` accepte un argument booléen facultatif qui indique si le chargement paresseux doit être empêché. Par exemple, vous pouvez souhaiter désactiver le chargement paresseux uniquement dans les environnements de non-production afin que votre environnement de production continue à fonctionner normalement même si une relation chargée paresseusement est accidentellement présente dans le code de production. Typiquement, cette méthode devrait être invoquée dans la méthode de démarrage de l'AppServiceProvider de votre application :

In [ ]:
%%php 

use Illuminate\Database\Eloquent\Model ;
 
/**
 * Bootstrap de tout service d'application.
 *
 * @return void
 */
public function boot()
{
    Model::preventLazyLoading( ! $this->app->isProduction()) ;
}

Vous pouvez également demander à Laravel de lever une exception lors de la tentative de remplissage d'un attribut non remplissable en invoquant la méthode preventSilentlyDiscardingAttributes. Cela permet d'éviter des erreurs inattendues au cours du développement local lorsque l'on tente de définir un attribut qui n'a pas été ajouté au tableau remplissable du modèle :

In [ ]:
%%php

Model::preventSilentlyDiscardingAttributes( ! $this->app->isProduction()) ;

Enfin, vous pouvez demander à Eloquent de lever une exception si vous tentez d'accéder à un attribut sur un modèle alors que cet attribut n'a pas été récupéré dans la base de données ou que l'attribut n'existe pas. Par exemple, cela peut se produire lorsque vous oubliez d'ajouter un attribut à la clause de sélection d'une requête Eloquent :

In [ ]:
%%php

Model::preventAccessingMissingAttributes( ! $this->app->isProduction()) ;

##### Activation du "mode strict" d'Eloquent

Pour des raisons pratiques, vous pouvez activer les trois méthodes mentionnées ci-dessus en invoquant simplement la méthode `shouldBeStrict` :

In [ ]:
%%php 

Model::shouldBeStrict( ! $this->app->isProduction()) ;

### Propriétés du modèle

Jusqu'à présent, nous n'avons parlé de la classe de modèle que sous l'angle de la configuration.

En effet, contrairement à `Symfony`, les propriétés ne sont pas définies _dans_ la classe d'entité (ou de modèle). Elles le sont dans la classe de migration associée à l'entité. C'est pourquoi vous aurez toujours intérêt à créer les classes de modèle avec l'option `--migration` qui crée la classe de migration.

En vérité, la solution la plus simple et la plus sûre est de créer la classe de modèle avec tous ses satellites :

In [ ]:
%%bash

./artisan create:model Product --all

## Associations entre entités du modèle

### Introduction

Contrairement aux propriétés, les associations sont décrites dans la classe de modèle.

### One-to-one

L'association **OneToOne** est sans doute la moins fréquente des associations. Parmi les exemples couramment évoqués, on pense à la relation entre client et panier sur un site de commerce en ligne.

Ce type d'association est souvent éliminé au profit de la fusion des deux tables, dans une base de données relationnelle, mais elle peut servir à indiquer des dépendances optionnelles (ou « _non rigides_ »). Je peux avoir un profil client sur un site sans pour autant acheter quelque chose.

Dans ce cas, Laravel prévoit une méthode `hasOne`.

In [ ]:
namespace App\Models;
 
use Illuminate\Database\Eloquent\Model;
use Illuminate\Database\Eloquent\Relations\HasOne;
 
class User extends Model
{
    /**
     * Un panier associé à un utilisateur
     */
    public function phone(): HasOne
    {
        return $this->hasOne(Cart::class);
    }
}

On notera que si l'association est spécifée, les noms de propriétés correspondantes ne le sont pas. Par défaut, `Laravel` les déduira du contexte. Ici par exemple, il prendra pour nom de colonne support de la clef étrangère `cart_id` et suppposera que sa cible est une colonne `id` (clef primaire) de la table `cart`.

Mais naturellement, nous pouvons modifier ce comportement par défaut en ajoutant les noms de colonnes dans la définition.

Autre point, chaque association peut avoir une association inverse, qui n'est dependant pas obligatoire. Dans ce cas, nous aurions à définir dans le modèle `Cart` une fonction `user()` avec la méthode `belongsTo()`.

Nous voyons au passage que les assocviations ne sont pas symétriques, véritablement, et qu'il y a toujours un sens privilégié qui peut avoir de l'importance.

In [ ]:
namespace App\Models;
 
use Illuminate\Database\Eloquent\Model;
use Illuminate\Database\Eloquent\Relations\BelongsTo;
 
class Cart extends Model
{
    /**
     * Un utilisateur associé à un panier
     */
    public function user(): BelongsTo
    {
        return $this->belongsTo(Cart::class);
    }
}

### One-To-Many

L'association **OneToMany** définit l'association d'une liste d'objet à un unique objet. Dans les spécifications UML, cette association prend deux formes possibles : **agrégation** ou **composition**.

> **Note* La composition est une association plus forte puisqu'elle signifie que le objets sont _intrinsèquement_ liés entre eux et que leurexistence séparée n'aurait pas de sens. Ce qui veut dire concrètemeent que supprimer l'un entraînerait de supprimer les autres.

Pour faire cela, `Laravel` propose une méthode `hasMany`, qui est l'association _directrice_.

> **Note** On remarquera que le choix ets inverse de celui fait par `Doctrine`

Typiquement, nos produits sont classés en « types de produits » :

In [ ]:
%%php 

namespace App\Models;
 
use Illuminate\Database\Eloquent\Model;
use Illuminate\Database\Eloquent\Relations\HasMany;
 
class ProductType extends Model
{
    /**
     * Un panier associé à un utilisateur
     */
    public function products(): HasMany
    {
        return $this->hasMany(Product::class);
    }
}

De la même manière que précédemment, lassociation invers, **ManyToOne** sera implémentée par la méthode `belongsTo()`.

#### Méthodes avancées

Les classes de modèle permettent aussi d'implémenter des accesseurs qui nous donneront accès à _un_ objet privilégié parmi les « _many_ ». Grâce à cela,, nous aurons l'impression d'une association OnrToOne implicite.

Quelques exemples :

In [ ]:
// Le dernier produit enregistré de cette catégorie
public function latestProduct(): HasOne
{
    return $this->hasOne(Product::class)->latestOfMany();
}

// Le plus ancien produit enregistré de cette catégorie
public function latestProduct(): HasOne
{
    return $this->hasOne(Product::class)->oldestOfMany();
}

// Le produit de la catégorie dont le prix est le plus petit
public function latestProduct(): HasOne
{
    return $this->hasOne(Product::class)->ofMany('price', min);
}

// Le dernier produit enregistré de cette catégorie
public function latestProduct(): HasOne
{
    return $this->hasOne(Product::class)->latestOfMany();
}

// Le produit le plus cher de cette catégorie qui soit actuellement en stock
public function currentPricing(): HasOne
{
    return $this->hasOne(Product::class)->ofMany([
        'price' => 'max',
    ], function (Builder $query) {
        $query->where('quantityInStock', '>', 0);
    });
}

Les modèles `Laravel`savent aussi gérer des chaînes d'association.

Dans notre cas, par exemple, si un `Product` est lié à une `Producer`, alors il est possible d'implémenter une méthode dans la classe `Productype` quinous donne directement accès au producteur.

### Many-to-Many

L'association **ManyToMany** est la plus générale puisqu'elle permet es associations multiples réciproques. Le vcas typique es la notion d'ami. Chaque personne a plusieurs amis, qui ont eux-mêmes plusieurs amis.

Ou dans notre cas, un utilisateur peut avoir plusieurs rôles et un rôle peut être tenu par plusieurs utilisateurs.

`Laravel` fournit pour cela une méthode `belongsToMany()` qui, cette fois-ci est symétrique.

In [ ]:
namespace App\Models;
 
use Illuminate\Database\Eloquent\Model;
use Illuminate\Database\Eloquent\Relations\BelongsToMany;
 
class User extends Model
{
    /**
     * The roles that belong to the user.
     */
    public function roles(): BelongsToMany
    {
        return $this->belongsToMany(Role::class);
    }
}

Généralement, nous créerons la méthode en miroir dans le modèle `Role`.

#### Table de liaison

Comme `SQL` ne sait pas gérer ce genre d'association, il faut passer par une table intermédiaire, appelée table d jointure ou table de liaison.

C'est `Laravel` qui s'en occupe et créera automatiquement une table (ici `role_user`) contenant deux clefs étrangères vers les tables associées. 

> **Note** Cette table a aussi des colonnes de datation, ce qui permet par la suite de faire des requêtes prenant en compte cette information.

## Associations polymorphes

Dans certains cas, les objets associés à un objet donné **ne sont pas du même type**.

> **Note** `Doctrine` résout ce problème en implémentant un héritage entre les entités, au même titre que dans la POO

`Laravel` adopte comme solution de changer de type d'association.

Par exemple, pour **OneToOne**, la méthode `hasOne()` est remplacée par les méthodes `morphTo()` et `morphOne()`. A titre d'exemple, si nous avons une banque d'image, chacune d'elles pouvant être relative à un produit ou à une personne (exclusivement, c'est important ici), nous pourrions avoir la structure de classes suvante :

In [ ]:
%%php

namespace App\Models;
 
use Illuminate\Database\Eloquent\Model;
use Illuminate\Database\Eloquent\Relations\MorphTo;
 
class Image extends Model
{
    /**
     * Associer un produit ou un utilisateut
     */
    public function imageable(): MorphTo
    {
        return $this->morphTo();
    }
}
 
use Illuminate\Database\Eloquent\Model;
use Illuminate\Database\Eloquent\Relations\MorphOne;
 
class Product extends Model
{
    /**
     * Associer une image à un produit
     */
    public function image(): MorphOne
    {
        return $this->morphOne(Image::class, 'imageable');
    }
}
 
use Illuminate\Database\Eloquent\Model;
use Illuminate\Database\Eloquent\Relations\MorphOne;
 
class User extends Model
{
    /**
     * Associer une image à un utilisateur.
     */
    public function image(): MorphOne
    {
        return $this->morphOne(Image::class, 'imageable');
    }
}

- Pour les associations **OneToMany**, les équivalents sont `morphTo()`et `morphMany()`
- Pour les association **ManytoMany**,  les équivalents sont `morphToMany()`et `morphByMany()`

## Requêtes simples

Une fois que vous avez créé un modèle et sa table de base de données associée, vous êtes prêt à commencer à extraire des données de votre base de données. Vous pouvez considérer chaque modèle Eloquent comme un puissant générateur de requêtes vous permettant d'interroger couramment la table de base de données associée au modèle. La méthode all du modèle récupère tous les enregistrements de la table de base de données associée au modèle :

In [ ]:
%%php

use App\Models\Product ;
 
foreach (Product::all() as $product) {
    echo $flight->name ;
}

#### Création de requêtes

La méthode Eloquent `all` renvoie tous les résultats dans le tableau du modèle. Cependant, puisque chaque modèle Eloquent sert de constructeur de requêtes, vous pouvez ajouter des contraintes supplémentaires aux requêtes et ensuite invoquer la méthode get pour récupérer les résultats :

In [ ]:
%%php

$vols = Flight::where('active', 1)
               ->orderBy('name')
               ->take(10)
               ->get() ;